In [1]:
# Importing Qiskit
from qiskit import *
from qiskit.qasm2 import dumps
from qiskit_aer import Aer

import numpy as np
import random
from random import randrange

In [2]:
def NoisyChannel(qc1, qc2, qc1_name, noise = 0.05):
    ''' This function takes the output of a circuit qc1 (made up only of x and 
        h gates), simulates a noisy quantum channel where Pauli errors (X - bit flip; Z - phase flip)
        will occur in qc2, and then initializes another circuit qc2 with the introduced noise.
    ''' 
    
    # Retrieve quantum state from qasm code of qc1
    qs = [dumps(qc1[i]).split('\n') for i in range(len(qc1))]
    
    # Process the code to get the instructions
    parsed_instructions = []
    for i, qasm_code in enumerate(qs):
        for line in qasm_code:
            line = line.strip()    # removing leading/trailing whitespace
            if line.startswith(('x', 'h', 'measure')):
                line = line.replace('0', str(i))
                parsed_instructions.append(line)
    
    # Apply parsed instructions to qc2
    for instruction in parsed_instructions:
        if instruction.startswith('x'):
            old_qr = int(instruction.split()[1][2:-2])
            qc2[old_qr].x(0)
            
        elif instruction.startswith('h'):
            old_qr = int(instruction.split()[1][2:-2])
            qc2[old_qr].h(0)
        
        elif instruction.startswith('measure'):
            continue    # exclude measuring
            
        else:
            print(f"Unable to parse instruction: {instruction}")
            raise Exception('Unable to parse instruction')
    
    # Introducing noise (taking input)
    for instruction in parsed_instructions:
        if random.random() < noise:
            old_qr = int(instruction.split()[1][2:-2])
            qc2[old_qr].x(0)     # Apply bit-flip error
            
        if random.random() < noise:
            old_qr = int(instruction.split()[1][2:-2])
            qc2[old_qr].z(0)     # Apply phase-flip error

In [3]:
def generate_random_bits(num):
    """This function generates a random bit-string of size = num"""
    # bits = np.array([random.randint(0, 1) for _ in range(num)])    # Randomly fills the array with 0/1
    
    bit_string = ""
    for _ in range(num):
        rand_bit = random.randint(0, 1)     # Flip Coin
        bit_string += str(rand_bit)
        
    return bit_string
    # return bits

In [4]:
def generate_random_bases(num_of_bases):
    """This function selects a random basis for each bit"""
    
    bases_string = ""
    for _ in range(num_of_bases):
        randBasis = random.randint(0, 1)     # Flip Coin

        if randBasis == 0:
            bases_string += "Z" 
        else:
            bases_string += "X"
            
    return bases_string

In [5]:
def encode(bits, bases):
    """This function encodes each bit into the given basis."""
    
    encoded_qubits = []
    
    for bit, basis in zip(bits, bases):
        qc = QuantumCircuit(1, 1)     # Create a quantum circuit for each qubit
        
        # Possible Cases
        if bit == "1" :
            qc.x(0)

        if basis == 'X' :
            qc.h(0)
            
        encoded_qubits.append(qc)
            
    return (encoded_qubits)

In [6]:
def measure(qubits, bases):
    """This function measures each qubit in the corresponding basis chosen for it."""

    bits = ""    # The results of measurements

    for qubit, basis in zip(qubits, bases):

        if basis == "X" :
            qubit.h(0)
            
        qubit.measure(0, 0)
        
        # Execute on Simulator
        simulator = Aer.get_backend('qasm_simulator')
        transpiled_circuit = transpile(qubit, simulator)
        result = simulator.run(transpiled_circuit, shots=1).result()
        counts = result.get_counts()
        measured_bit = max(counts, key=counts.get)     # Max doesn't matter for simulator since there is only one shot.

        bits += str(measured_bit)
        
    return bits

# Start

In [7]:
eve_presence = 0
KEY_LENGTH = 500
ch_noise = 0.001
error_threshold = 0.2

# Preparation for encoding
random.seed(0)    # Seed the random number generator. This will be used as our "coin flipper"

## Alice

In [8]:
# Generating a random string of bits
alice_bits = generate_random_bits(KEY_LENGTH)
alice_bases = generate_random_bases(KEY_LENGTH) # Alice randomly chooses a basis for each bit.
alice_bases[:100]

'ZXXZZZZXZXXXZZZXXXXXXZZZXZXZZZXZZZXXXZZZXZZZXXXZXXXZZXXZXXZXXXZXXXZZZXZZXXZXXZZXXZXXZXZXZXXZZXZXZXXZ'

### Encoding

In [9]:
# Encode Alice's bits
encoded_qubits = encode(alice_bits, alice_bases)

# Quantum Signal Channel

## Eve
0 : Not present,    1 : present

In [10]:
if eve_presence == 'Random': eve = random.randint(0, 1)
else: eve = int(eve_presence)
    
label = 'Eve' if eve else 'Alice'

In [11]:
qubits_received = [QuantumCircuit(1, 1) for _ in range(len(encoded_qubits))]

if eve : 
    #print("Eve Present!")
    qubits_intercepted = [QuantumCircuit(1, 1) for _ in range(len(encoded_qubits))]
    
    NoisyChannel(encoded_qubits, qubits_intercepted, 'alice', noise = ch_noise) ##Eve intercepts noisy states     

    eve_bases = generate_random_bases(KEY_LENGTH) # Generate a random set of bases
    eve_bits = measure(qubits_intercepted, eve_bases) # Measure the qubits
    
    # Eve encodes her decoy qubits and sends them along the quantum channel    
    encoded_intercepted_qubits = encode(eve_bits, eve_bases)    
    NoisyChannel(encoded_intercepted_qubits, qubits_received, 'Eve', noise = ch_noise) ## Eve sends noisy states to Bob

else : 
    NoisyChannel(encoded_qubits, qubits_received, 'Alice', noise = ch_noise) ## Alice sends noisy states to Bob


## Bob

In [12]:
bob_bases = generate_random_bases(KEY_LENGTH) # Bob randomly chooses a basis for each bit.
print(bob_bases[:100])

# Measurement
bob_bits = measure(qubits_received, bob_bases)

ZZXZZZZXXZZZZXZXXXXZXZZZZXXZXZXZZXZXZZZXZZZXXXZZXZXXXXXXZXZZXXXXZXZZZZXXZXXXZXZZZZZZXXXZXZZXZZZXZZXZ


In [13]:
bob_bits[:100]

'1001111111110010100110110011000011111101100000110111101011101010011001010110011101011111001000011000'

# Public Interaction Channel

In [14]:
BROADCAST = alice_bases    # Alice tells Bob which bases she used. BROADCAST uses classical channel

# Store the indices of the bases they share in common
common_bases = [i for i in range(KEY_LENGTH) if BROADCAST[i] == bob_bases[i]]

In [15]:
for idx, item in enumerate(zip(bob_bases, alice_bases)):
    if bob_bases[idx] == alice_bases[idx]:
        print(idx, item, (bob_bits[idx], alice_bits[idx]))

0 ('Z', 'Z') ('1', '1')
2 ('X', 'X') ('0', '0')
3 ('Z', 'Z') ('1', '1')
4 ('Z', 'Z') ('1', '1')
5 ('Z', 'Z') ('1', '1')
6 ('Z', 'Z') ('1', '1')
7 ('X', 'X') ('1', '1')
12 ('Z', 'Z') ('0', '0')
14 ('Z', 'Z') ('1', '1')
15 ('X', 'X') ('0', '0')
16 ('X', 'X') ('1', '1')
17 ('X', 'X') ('0', '0')
18 ('X', 'X') ('0', '0')
20 ('X', 'X') ('1', '1')
21 ('Z', 'Z') ('0', '0')
22 ('Z', 'Z') ('1', '1')
23 ('Z', 'Z') ('1', '1')
26 ('X', 'X') ('1', '1')
27 ('Z', 'Z') ('1', '1')
29 ('Z', 'Z') ('0', '0')
30 ('X', 'X') ('0', '0')
31 ('Z', 'Z') ('0', '0')
32 ('Z', 'Z') ('1', '1')
35 ('X', 'X') ('1', '1')
37 ('Z', 'Z') ('1', '1')
38 ('Z', 'Z') ('0', '0')
41 ('Z', 'Z') ('0', '0')
42 ('Z', 'Z') ('0', '0')
44 ('X', 'X') ('0', '0')
45 ('X', 'X') ('0', '0')
47 ('Z', 'Z') ('1', '1')
48 ('X', 'X') ('0', '0')
50 ('X', 'X') ('1', '1')
53 ('X', 'X') ('0', '0')
54 ('X', 'X') ('1', '1')
57 ('X', 'X') ('1', '1')
58 ('Z', 'Z') ('1', '1')
60 ('X', 'X') ('1', '1')
61 ('X', 'X') ('0', '0')
63 ('X', 'X') ('0', '0')
65 ('X'

In [16]:
print(common_bases[:100])

[0, 2, 3, 4, 5, 6, 7, 12, 14, 15, 16, 17, 18, 20, 21, 22, 23, 26, 27, 29, 30, 31, 32, 35, 37, 38, 41, 42, 44, 45, 47, 48, 50, 53, 54, 57, 58, 60, 61, 63, 65, 66, 67, 68, 73, 75, 78, 81, 85, 92, 94, 95, 96, 98, 99, 100, 102, 105, 107, 108, 110, 111, 112, 115, 117, 118, 119, 124, 126, 129, 131, 133, 134, 135, 136, 137, 138, 140, 144, 145, 146, 147, 150, 151, 152, 161, 165, 166, 167, 168, 169, 171, 174, 175, 178, 180, 182, 183, 184, 186]


In [17]:
bob_bits = [bob_bits[index] for index in common_bases]
BROADCAST = common_bases    # Bob tells Alice which bases they shared in common
alice_bits = [alice_bits[index] for index in BROADCAST]    # Alice keeps only the bits they shared in common

In [18]:
bob_bits[:100]

['1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '0',
 '1',
 '0',
 '0',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '0',
 '0',
 '0',
 '1',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '0',
 '0',
 '1',
 '1',
 '0',
 '0',
 '1',
 '0',
 '1',
 '1',
 '1',
 '0',
 '0',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '1',
 '0',
 '1',
 '0',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '0',
 '0',
 '0',
 '1',
 '1',
 '0',
 '1',
 '0']

In [19]:
alice_bits[:100]

['1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '0',
 '1',
 '0',
 '0',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '0',
 '0',
 '0',
 '1',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '0',
 '0',
 '1',
 '1',
 '0',
 '0',
 '1',
 '0',
 '1',
 '1',
 '1',
 '0',
 '0',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '1',
 '0',
 '1',
 '0',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '0',
 '0',
 '0',
 '1',
 '1',
 '0',
 '1',
 '0']

## Comparision (Spotting)

In [20]:
sample = len(alice_bits)//3    # len(alice_bits) >= 3
errors = 0

for _ in range(sample):
    bit_index = random.randrange(len(alice_bits)) 
    
    if alice_bits[bit_index] != bob_bits[bit_index]:  errors += 1    #calculating errors
        
    del alice_bits[bit_index] #removing tested bits from key strings
    del bob_bits[bit_index]

# QBER

In [21]:
QBER = round(errors/sample, 4) # calculating QBER and saving the answer to two decimal places
QBER

0.0

## Final Key : 

In [22]:
if QBER > error_threshold:
    if eve : print(" Eve detected : ", end = " ")
    else : print(" Eve FALSELY not detected : ", end = " ")
    print("Key not secure")

else :
    if eve : print('Eve went unnoticed : ', end = " ")
    else : print('Eve not present : ', end = " ")
    print("Key is secure")


Eve not present :  Key is secure


In [23]:
key = "" 
for bit in alice_bits:    # Or bob_bits, since both should be the same
    key += bit

In [24]:
errors, sample

(0, 86)